In [ ]:
# Clone the repository
!git clone https://github.com/DebadityaHait/PINNs
%cd PINNs

# Install required packages
%pip install torch numpy matplotlib pandas scipy seaborn


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from IPython.display import display, HTML

# Add src directory to path for deepxdeAbeta imports
sys.path.append('./src')

# Create necessary directories if they don't exist
os.makedirs('data', exist_ok=True)
os.makedirs('results', exist_ok=True)

# Check if deepxdeAbeta is accessible
try:
    import deepxdeAbeta.deepxde as dde
    print("✅ deepxdeAbeta imported successfully")
except ImportError as e:
    print(f"❌ Error importing deepxdeAbeta: {e}")


In [ ]:
def load_model_data(model_num):
    """Load model data from results directory"""
    model_dir = f"results/models/model{model_num}"
    
    # Check if model directory exists
    if not os.path.exists(model_dir):
        print(f"Model {model_num} directory not found: {model_dir}")
        return None
    
    # Load data
    try:
        info = pd.read_csv(f"{model_dir}/info.dat", sep='\s+', header=None)
        variables = pd.read_csv(f"{model_dir}/variables.dat", sep='\s+', header=None)
        loss = pd.read_csv(f"{model_dir}/loss.dat", sep='\s+', header=None)
        train = pd.read_csv(f"{model_dir}/train.dat", sep='\s+', header=None)
        test = pd.read_csv(f"{model_dir}/test.dat", sep='\s+', header=None)
        
        # Load model weights if needed
        model_file = f"{model_dir}/nonneg_constr-35000.pt"
        if not os.path.exists(model_file):
            model_file = f"{model_dir}/nonneg_constr-50000.pt"  # Model 8 uses 50000
        
        model_exists = os.path.exists(model_file)
        
        return {
            "info": info,
            "variables": variables,
            "loss": loss,
            "train": train,
            "test": test,
            "model_file": model_file if model_exists else None,
            "model_exists": model_exists
        }
    except Exception as e:
        print(f"Error loading data for model {model_num}: {e}")
        return None

def plot_loss_history(model_num, model_data):
    """Plot loss history for a model"""
    if model_data is None or "loss" not in model_data:
        print(f"No loss data available for model {model_num}")
        return
    
    loss_df = model_data["loss"]
    
    plt.figure(figsize=(10, 6))
    plt.semilogy(loss_df[0], loss_df[1], label='Loss')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.title(f'Model {model_num} Loss History')
    plt.legend()
    plt.grid(True, which="both", ls="-")
    plt.show()

def plot_parameter_evolution(model_num, model_data):
    """Plot parameter evolution for a model"""
    if model_data is None or "variables" not in model_data:
        print(f"No variable data available for model {model_num}")
        return
    
    var_df = model_data["variables"]
    
    # Assuming column 0 is iteration, and the rest are parameters
    plt.figure(figsize=(12, 8))
    for i in range(1, len(var_df.columns)):
        plt.plot(var_df[0], var_df[i], label=f'Parameter {i}')
    
    plt.xlabel('Iterations')
    plt.ylabel('Parameter Value')
    plt.title(f'Model {model_num} Parameter Evolution')
    plt.legend()
    plt.grid(True)
    plt.show()

def display_final_parameters(model_data_dict):
    """Display final parameters for all models"""
    # Create a DataFrame to hold parameters from all models
    final_params = {}
    param_names = ["k1", "k2", "k3", "k4", "k5", "k6", "k7", "k8", "k9", "k10", "k11", "k12"]
    
    for model_num, data in model_data_dict.items():
        if data is None or "variables" not in data:
            continue
            
        # Get the last row of variables (final parameters)
        var_df = data["variables"]
        final_row = var_df.iloc[-1]
        
        # Skip the first column (iteration number)
        params = final_row[1:].values
        final_params[f"Model {model_num}"] = params
    
    # Create DataFrame with parameter names
    num_params = max([len(p) for p in final_params.values()]) if final_params else 0
    param_names = param_names[:num_params]
    
    df = pd.DataFrame(final_params)
    df.index = param_names[:len(df)]
    
    # Display the table
    display(HTML("<h3>Final Parameter Values for All Models</h3>"))
    display(df)
    
    # Save to CSV
    df.to_csv("final_parameters.csv")
    print("Final parameters saved to final_parameters.csv")
    
    return df


In [ ]:
def run_model(model_num):
    """Run a specific model by executing its notebook"""
    print(f"Checking if model {model_num} results exist...")
    
    model_dir = f"results/models/model{model_num}"
    if os.path.exists(model_dir) and os.path.exists(f"{model_dir}/variables.dat"):
        print(f"Model {model_num} results already exist. Skipping execution.")
        return True
    
    print(f"Running model {model_num}...")
    try:
        # For Colab, we need to get the notebook content from the repository
        notebook_path = f"notebooks/rompinnmodels_model{model_num}.ipynb"
        
        if not os.path.exists(notebook_path):
            print(f"Notebook {notebook_path} not found.")
            return False
        
        # Create a temporary Python script from the notebook
        import subprocess
        subprocess.run(["jupyter", "nbconvert", "--to", "python", notebook_path, "--output", f"temp_model{model_num}"])
        
        # Modify the script to add necessary imports and path adjustments
        with open(f"temp_model{model_num}.py", "r") as f:
            script_content = f.read()
        
        # Add sys.path.append to ensure deepxdeAbeta can be imported
        modified_script = "import sys\nsys.path.append('./src')\n" + script_content
        
        # Replace any references to binning_trials with results
        modified_script = modified_script.replace("binning_trials/", "results/")
        
        with open(f"temp_model{model_num}.py", "w") as f:
            f.write(modified_script)
        
        # Run the modified script
        subprocess.run(["python", f"temp_model{model_num}.py"])
        
        # Clean up
        subprocess.run(["rm", f"temp_model{model_num}.py"])
        
        return True
    except Exception as e:
        print(f"Error running model {model_num}: {e}")
        return False


In [ ]:
# Run all models if needed
for model_num in range(1, 9):
    run_model(model_num)


In [ ]:
# Load data for all models
model_data_dict = {}
for model_num in range(1, 9):
    print(f"Loading data for model {model_num}...")
    model_data = load_model_data(model_num)
    if model_data is not None:
        model_data_dict[model_num] = model_data
        print(f"✅ Model {model_num} data loaded successfully")
    else:
        print(f"❌ Failed to load data for model {model_num}")

print(f"\nLoaded data for {len(model_data_dict)} models")


In [ ]:
# Plot loss history for each model
for model_num, model_data in model_data_dict.items():
    print(f"\n### Model {model_num} Loss History ###")
    plot_loss_history(model_num, model_data)


In [ ]:
# Plot parameter evolution for each model
for model_num, model_data in model_data_dict.items():
    print(f"\n### Model {model_num} Parameter Evolution ###")
    plot_parameter_evolution(model_num, model_data)


In [ ]:
# Display final parameters table for all models
final_params_df = display_final_parameters(model_data_dict)


In [ ]:
# Plot combined loss history for all models
plt.figure(figsize=(12, 8))

for model_num, model_data in model_data_dict.items():
    if model_data is None or "loss" not in model_data:
        continue
        
    loss_df = model_data["loss"]
    plt.semilogy(loss_df[0], loss_df[1], label=f'Model {model_num}')

plt.xlabel('Iterations')
plt.ylabel('Loss (log scale)')
plt.title('Combined Loss History for All Models')
plt.legend()
plt.grid(True, which="both", ls="-")
plt.show()


In [ ]:
# Create a heatmap of final parameters across models
import seaborn as sns

plt.figure(figsize=(14, 10))
sns.heatmap(final_params_df, annot=True, cmap="YlGnBu", fmt=".4f")
plt.title("Parameter Values Across Models")
plt.tight_layout()
plt.show()

# Save the figure
plt.figure(figsize=(14, 10))
sns.heatmap(final_params_df, annot=True, cmap="YlGnBu", fmt=".4f")
plt.title("Parameter Values Across Models")
plt.tight_layout()
plt.savefig("parameter_comparison_heatmap.png")
print("Saved parameter comparison heatmap to parameter_comparison_heatmap.png")


In [ ]:
# Compare final loss values across models
final_loss = {}

for model_num, model_data in model_data_dict.items():
    if model_data is None or "loss" not in model_data:
        continue
        
    loss_df = model_data["loss"]
    final_loss[f"Model {model_num}"] = loss_df[1].iloc[-1]

loss_comparison_df = pd.DataFrame(list(final_loss.items()))
loss_comparison_df.columns = ["Model", "Final Loss"]
loss_comparison_df = loss_comparison_df.sort_values("Final Loss")

plt.figure(figsize=(10, 6))
plt.bar(loss_comparison_df["Model"], loss_comparison_df["Final Loss"])
plt.xlabel("Model")
plt.ylabel("Final Loss Value")
plt.title("Final Loss Comparison Across Models")
plt.yscale("log")
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Add value labels on top of bars
for i, v in enumerate(loss_comparison_df["Final Loss"]):
    plt.text(i, v * 1.05, f"{v:.6f}", ha="center", va="bottom", rotation=90, fontsize=9)

plt.tight_layout()
plt.show()

# Save the figure
plt.figure(figsize=(10, 6))
plt.bar(loss_comparison_df["Model"], loss_comparison_df["Final Loss"])
plt.xlabel("Model")
plt.ylabel("Final Loss Value")
plt.title("Final Loss Comparison Across Models")
plt.yscale("log")
plt.grid(axis="y", linestyle="--", alpha=0.7)
for i, v in enumerate(loss_comparison_df["Final Loss"]):
    plt.text(i, v * 1.05, f"{v:.6f}", ha="center", va="bottom", rotation=90, fontsize=9)
plt.tight_layout()
plt.savefig("final_loss_comparison.png")
print("Saved final loss comparison to final_loss_comparison.png")


In [ ]:
# Display summary statistics
print("# Summary of Model Results")
print(f"Number of models analyzed: {len(model_data_dict)}")

if loss_comparison_df is not None and not loss_comparison_df.empty:
    best_model = loss_comparison_df.iloc[0]["Model"]
    best_loss = loss_comparison_df.iloc[0]["Final Loss"]
    print(f"Best performing model: {best_model} with final loss: {best_loss:.6f}")

print("\nFinal parameter values have been saved to final_parameters.csv")
print("Comparison figures have been saved as PNG files")
